In [50]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd

Crawl the URL and parse the HTML doc

In [7]:
res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = BeautifulSoup(res.content, 'html.parser')

In [47]:
data = []
table = soup.find('table', attrs={'class':'wikitable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    if len(cols) > 0:
        cols = [ele.text.strip() for ele in [x for x in cols if x != None and len(x) > 0]] # Get rid of the first fow which contains table header
        data.append([ele for ele in cols if ele])  

Generate the dataframe

In [78]:
df = pd.DataFrame(data)
df.columns = ['postcode', 'borough', 'neighborhood']
df.head()

,postcode,borough,neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Filter out the "Not assigned" boroughs

In [79]:
df_2 = df[df.borough != 'Not assigned']

Replace "Not assigned" neighborhoods using their boroughs

In [93]:
df_2.loc[df_2.neighborhood == 'Not assigned','neighborhood'] = df_2['borough']

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Check the data after cleaning

In [94]:
df_2.head()

,postcode,borough,neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [98]:
print('check the number of rows before and after data cleaning')
print('before cleaning: {} rows'.format(len(df)))
print('after cleaning: {} rows'.format(len(df_2)))

check the number of rows before and after data cleaning
before cleaning: 288 rows
after cleaning: 211 rows


In [96]:
df_2[df_2.postcode == 'M5A'].head()

,postcode,borough,neighborhood
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park


In [97]:
df_2.shape

(211, 3)